# Métodos iterativos para la álgebra lineal numérica

In [2]:
using Plots, LaTeXStrings
pyplot()

Plots.PyPlotBackend()

En los notebooks anteriores, vimos que podemos resolver EDPs lineales estacionarios al reducirlos, a través de una discretización en el espacio, a problemas de álgebra lineal, y vimos que hay **métodos directos** para resolverlos, como el método de eliminación Gaussiana (o factorización $LU$). Los métodos de esta índole se llaman "directos", ya que proveen un algoritmo que resuelve directamente el problema, con un algoritmo que garantizadamente termina después de un número de operaciones computacionales que se conoce de antemano.

**[1]** (Opcional.)

Para una matriz de tamaño $N \times N$, encuentra la *complejidad computacional* del algoritmo de eliminación Gaussiana, es decir, cómo escala el número total de operaciones requeridas con el tamaño $N$ del sistema.

Sin embargo, también vimos que el método de eliminación Gaussiana resulta a menudo ser muy poco eficiente, ya que la matriz en cuestión tiene muchos ceros. Decimos que la matriz es **sparse** (inglés), o dispersa, rala, escasa, etc. en español. 

Existen otra clase muy distinta de algoritmos en la álgebra lineal numérica, los **métodos iterativos**. Si pensamos en los problemas estacionarios como literalmente el estado estacionario de una ecuación de evolución (artificial), es natural pensar en resolverlos con métodos iterativos, como ya hicimos en el caso 1D.

## El método de relajación 

El algoritmo iterativo más sencillo sirve para resolver ecuaciones lineales de tipo Poisson:

$$\nabla^2 V(\mathbf{x}) = -\rho(\mathbf{x}).$$

Vimos en el notebook anterior que al discretizar el operador $\nabla^2$ en 2D en una red cuadrada (con $h_x = h_y = h$), nos lleva a un sistema de ecuaciones (una para cada punto $(i,j)$ de la malla) de la forma

$$u_{i,j+1} + u_{i,j-1} + u_{i-1,j} + u_{i+1,j} - 4 u_{ij} = - h^2 \rho(x_{ij}).$$

Podemos convertir esta ecuación en método iterativo al despejar $u_{ij}$ e introducir un tiempo artificial:

$$u_{ij}^{(t+1)} = \textstyle \frac{1}{4} \left[ u_{i,j+1}^{(t)} + u_{i,j-1}^{(t)} + u_{i-1,j}^{(t)} + u_{i+1,j}^{(t)} + h^2 \rho(x_{ij}) \right]. \quad (*)$$

Eso se llama el **método de Jacobi**. Se llama un método de **relajación**, ya que la solución "se relaja" hacia el estado estacionario. Nótese que es necesario contar con dos matrices: una al paso $t$, y una al paso $t+1$.

In [3]:
u = [1 2; 2 3]

2×2 Array{Int64,2}:
 1  2
 2  3

In [4]:
typeof(u)

Array{Int64,2}

**[2]** (i) Implementa el método de Jacobi y utilízalo para resolver los mismos problemas físicos que en el notebook 19. Nótese que la condición inicial es arbitraria, pero ¡las condiciones de frontera son cruciales!

In [ ]:
function Jacobi(M1::Array{Number, 2})
     M_nueva = matrix
    
    

In [5]:
function Metodo_Jacobi(V0, N, L, tol) 
    
# Definiendo Vij    
    V = V0
# Condiciones de frontera
    for j in 1:N
        V[1,j] = 0.0
        V[N,j] = 10.0
    end
    for i in 1:N
        V[i,1] = 0.0
        V[i,N] = 0.0
    end
    
#Solucionador
    k = 1 
    error = 2*tol
     
    while k<=1e8 && error>tol
        for i in 2:N-1
            for j in 2:N-1
                V[i,j] = 0.25*(V[i, j+1] + V[i,j-1] + V[i-1,j] + V[i+1,j]) 
                error = error + abs(V[i,j] - V0[i,j])
            k = k + 1
            end
        end
    end
    return V
end

Metodo_Jacobi (generic function with 1 method)

In [16]:
Metodo_Jacobi(rand(100,100), 10.0, 1, 1.0) 

LoadError: ArgumentError: invalid index: 1.0

(ii) ¿Está mejor este método o el de eliminación gaussiana para este problema? ¿Por qué?

(iii) ¿A cuál ecuación parcial diferencial de evolución corresponde la ecuación (*)?  De esta forma, ¿cómo podemos interpretar el método?

(iv) Calcula el estado estacionario (después de una evolución larga) y guárdala. Repite la evolución (desde la misma condición inicial) para calcular cómo cambia la distancia (en una norma adecuada) desde el estado estacionario en el tiempo, y grafícala. ¿Qué comportamiento tiene?

El método de Jacobi converge lentamente, y no siempre converge. Otra forma de ver al método de Jacobi es el siguiente.

Descompongamos la matriz $\mathsf{M}$ que especifica el sistema 
de ecuaciones lineales de interés como 

$$ \mathsf{M} =  \mathsf{D} + \mathsf{R},$$ 

donde $\mathsf{D}$ es la parte diagonal y $\mathsf{R}$ el resto. 

El resolver $\mathsf{M} \cdot \mathbf{x} = \mathbf{b}$ es, entonces, equivalente a resolver

$$\mathsf{D} \cdot \mathbf{x} = \mathbf{b} - \mathsf{R} \cdot \mathbf{x}.$$

El punto es que ahora $\mathsf{D}$, siendo diagonal, es fácil de invertir, y llegamos al método iterativo

$$\mathbf{x}^{(t+1)} = \mathsf{D}^{-1} \cdot (\mathbf{b} - \mathsf{R} \cdot \mathbf{x}^{(t)}).$$

## El método de Gauss--Seidel 

Un algoritmo más eficiente que el de Jacobi es el método de **Gauss--Seidel**. La diferencia radica en que los nuevos valores calculados se *sobreescriben en cada paso en la misma matriz* (en lugar de almacenarse en una matriz diferente). 

**[3]** Implementa el método de Gauss--Seidel, y compara la distancia desde el estado estacionario con el método de Jacobi.

Existe otro método, llamado SOR ("Successive Over-Relaxation") que combina los dos métodos y puede ser (bastante) más eficiente. [¡Proyecto final!]